In [1]:
from os.path import dirname, join, isfile, isdir
from datetime import datetime, timedelta
from os import makedirs
import pandas as pd
from simpledbf import Dbf5
from bulletin import __file__ as __root__
from bulletin.commom import static
from bulletin.commom.normalize import normalize_text, normalize_labels, normalize_number, normalize_municipios, normalize_igbe, normalize_hash, date_hash

In [2]:
class sivep:
    
    def __init__(self, pathfile=''):
        self.__source = None
        self.pathfile = pathfile
        self.database = join(dirname(__root__),'resources','database','sivep.pkl')
        self.errorspath = join('output','errors','sivep',datetime.today().strftime('%B_%Y'))

        if not isdir(self.errorspath):
            makedirs(self.errorspath)

        if not isdir(dirname(self.database)):
            makedirs(dirname(self.database))

    def read(self,pathfile, append=False):
        srag = Dbf5(pathfile, codec = 'cp860').to_dataframe()
        #srag = self.__normalize(srag)

        if isinstance(self.__source, pd.DataFrame) and append:
            self.__source = self.__source.append(srag, ignore_index=True)
        else:
            self.__source = srag
    
    def read_all_database_files(self):
        last_digit = ['0', '1']

        for x in last_digit:
            self.read(join('input','dbf',f"SRAG202{x}.dbf"), append=True)

    def load(self):
        try:
            self.__source = pd.read_pickle(self.database)
        except:
            raise Exception(f"Arquivo {self.database} não encontrado")

    def save(self, df=None):
        if isinstance(df, pd.DataFrame) and len(df) > 0:
            new_df = df
            self.__source = new_df
        elif isinstance(self.__source, pd.DataFrame) and len(self.__source) > 0:
            new_df = self.__source
        else:
            raise Exception('Não é possível salvar um DataFrame inexistente, realize a leitura antes ou passe como para esse método')

        new_df.to_pickle(self.database)
    
    def to_notifica(self):
        self.__source = self.__source[['DT_NOTIFIC', 'DT_SIN_PRI', 'SG_UF_NOT', 'CO_MUN_NOT', 'ID_UNIDADE',
                                       'NU_CPF', 'NM_PACIENT', 'CS_SEXO', 'DT_NASC', 'NU_IDADE_N', 'TP_IDADE', 'CS_GESTANT',
                                       'CS_RACA', 'CS_ETINIA', 'NM_MAE_PAC', 'NU_CEP', 'ID_PAIS', 'SG_UF',
                                       'CO_MUN_RES', 'NM_BAIRRO', 'NM_LOGRADO', 'NU_NUMERO', 'FEBRE', 'TOSSE',
                                       'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO',
                                       'OUTRO_SIN', 'PUERPERA', 'CARDIOPATI', 'SIND_DOWN', 'HEPATICA', 'DIABETES',
                                       'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'OUT_MORBI',
                                       'ANTIVIRAL', 'TP_ANTIVIR', 'HOSPITAL', 'UTI', 'RAIOX_RES', 'AMOSTRA', 'PCR_RESUL',
                                       'DT_COLETA', 'REQUI_GAL', 'DT_PCR', 'POS_PCRFLU', 'TP_FLU_PCR', 'PCR_FLUASU',
                                       'FLUASU_OUT', 'PCR_FLUBLI', 'FLUBLI_OUT', 'POS_PCROUT', 'PCR_VSR', 'PCR_PARA1',
                                       'PCR_PARA2', 'PCR_PARA3', 'PCR_PARA4', 'PCR_ADENO', 'PCR_METAP', 'PCR_BOCA',
                                       'PCR_RINO', 'PCR_OUTRO', 'LAB_PCR', 'CLASSI_FIN', 'CRITERIO', 'EVOLUCAO',
                                       'DT_EVOLUCA', 'DT_ENCERRA', 'NOME_PROF', 'HISTO_VGM', 'LO_PS_VGM', 'DT_VGM',
                                       'DT_RT_VGM', 'PCR_SARS2', 'PAC_COCBO', 'PERD_OLFT', 'PERD_PALA', 'TOMO_RES',
                                       'POS_AN_FLU', 'TP_FLU_AN', 'POS_AN_OUT', 'AN_SARS2', 'AN_VSR', 'AN_PARA1',
                                       'AN_PARA2', 'AN_PARA3', 'AN_ADENO', 'AN_OUTRO', 'NU_DO']]

        self.__source = self.__source.rename(
                                    columns={
                                        'DT_NOTIFIC': 'data_notificacao',
                                        'DT_SIN_PRI': 'data_1o_sintomas',
                                        'SG_UF_NOT': 'uf_unidade_notifica',
                                        'CO_MUN_NOT': 'ibge_unidade_notifica',
                                        'ID_UNIDADE': 'nome_unidade_notifica',
                                        'NU_CPF': 'cpf',
                                        'NM_PACIENT': 'paciente',
                                        'CS_SEXO': 'sexo',
                                        'DT_NASC': 'data_nascimento',
                                        'CS_RACA': 'raca_cor',
                                        'CS_ETINIA': 'etnia',
                                        'NM_MAE_PAC': 'nome_mae',
                                        'NU_CEP': 'cep_residencia',
                                        'ID_PAIS': 'pais_residencia',
                                        'SG_UF': 'uf_residencia',
                                        'CO_MUN_RES': 'ibge_residencia',
                                        'NM_BAIRRO': 'bairro_residencia',
                                        'NM_LOGRADO': 'logradouro_residencia',
                                        'NU_NUMERO': 'numero_residencia',
                                        'FEBRE': 'febre',
                                        'TOSSE': 'tosse',
                                        'GARGANTA': 'dor_garganta',
                                        'DISPNEIA': 'dispneia',
                                        'DESC_RESP': 'dispneia',
                                        'SATURACAO': 'saturacao_o2',
                                        'DIARREIA': 'diarreia',
                                        'VOMITO': 'nausea_vomitos',
                                        'OUTRO_SIN': 'outros_sintomas',
                                        'PUERPERA': 'puerperio',
                                        'CARDIOPATI': 'doenca_cardiovascular',
                                        'SIND_DOWN': 'sindrome_down',
                                        'HEPATICA': 'doenca_hepatica',
                                        'DIABETES': 'diabetes',
                                        'NEUROLOGIC': 'doenca_neurologica',
                                        'PNEUMOPATI': 'doenca_pulmonar',
                                        'IMUNODEPRE': 'imunodeficiencia',
                                        'RENAL': 'doenca_renal',
                                        'OBESIDADE': 'obesidade',
                                        'OUT_MORBI': 'outras_morbidades',
                                        'HOSPITAL': 'hospitalizado',
                                        'AMOSTRA': 'coleta_amostra',
                                        'DT_COLETA': 'data_coleta',
                                        'REQUI_GAL': 'requisicao',
                                        'DT_PCR': 'data_liberacao',
                                        'DT_EVOLUCA': 'data_cura_obito',
                                        'DT_ENCERRA': 'data_encerramento',
                                        'NOME_PROF': 'nome_notificador',
                                        'HISTO_VGM': 'historico_viagem',
                                        'LO_PS_VGM': 'local_viagem',
                                        'DT_VGM': 'data_ida_local',
                                        'DT_RT_VGM': 'data_retorno_local',
                                        'PERD_OLFT': 'perda_olfato_paladar',
                                        'PERD_PALA': 'perda_olfato_paladar',
                                        'NU_DO': 'numero_do',
                                        'PAC_COCBO': 'cod_cbo'
                                    })
        self.__source

In [3]:
sivep()